In [57]:
import sys
import time
import pandas as pd
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv
import ast

In [58]:
CONFLICT_PROMPT_RYU = """
You are an expert network engineer analyzing two Ryu (OpenFlow) flow rules to decide whether they **conflict**.

### **Primary Goal**
A conflict exists if the two rules apply to the **same switch (dpid)** and **same table_id**, their **matches overlap** (can match the same packet), AND at least one of the following is true:
1.  Their **actions are different** (e.g., `OUTPUT:1` vs. `OUTPUT:2`, or `[]` vs. `OUTPUT:1`).
2.  They are **redundant** (they overlap and have the **same action**).

### **Guiding Principles**
- If the `dpid` or `table_id` are different, they do **not** conflict.
- If the matches are **mutually exclusive** (e.g., they match on different L2 protocols like `LLDP` vs. `IPv4`, or different ports like `in_port: 1` vs `in_port: 2`), they do **not** conflict.

---
### **Examples**

**Example 1: Conflict (Different Action)**
Flow 1: {"dpid": 1, "priority": 100, "match": {"eth_type": 2048}, "actions": ["OUTPUT:1"]}
Flow 2: {"dpid": 1, "priority": 200, "match": {"eth_type": 2048, "ip_proto": 6}, "actions": ["OUTPUT:2"]}
Output: {
    "conflict_status": 1,
    "conflict_explanation": "Rules overlap (Flow 2 is a subset of Flow 1) but have different OUTPUT actions."
}

**Example 2: No Conflict (Mutual Exclusion)**
Flow 1: {"dpid": 4, "priority": 100, "match": {"eth_type": 2048, "ipv4_dst": "10.0.1.4"}, "actions": ["OUTPUT:4"]}
Flow 2: {"dpid": 4, "priority": 65535, "match": {"dl_dst": "01:80:c2:00:00:00"}, "actions": ["OUTPUT:CONTROLLER"]}
Output: {
    "conflict_status": 0,
    "conflict_explanation": "Matches are mutually exclusive. Flow 1 matches IPv4 packets (eth_type 2048), while Flow 2 matches a non-IPv4 L2 protocol (LLDP)."
}

**Example 3: Conflict (Redundancy / Same Action)**
Flow 1: {"dpid": 1, "priority": 100, "match": {"eth_type": 2048, "ip_proto": 6}, "actions": ["OUTPUT:1"]}
Flow 2: {"dpid": 1, "priority": 90, "match": {"eth_type": 2048}, "actions": ["OUTPUT:1"]}
Output: {
    "conflict_status": 1,
    "conflict_explanation": "Rules are redundant. The matches overlap (Flow 1 is a subset of Flow 2) and they have the same action."
}
---

### **Input Format**
You will be provided with **two JSON flow rules** in the following format:

**Flow 1:**
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - 'conflict_status' should be 1 if a conflict exists, 0 otherwise.
   - 'conflict_explanation' is a brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
"""

In [59]:
# model names should match the downloaded ollama model names. 
# If ollama does not find a model, check ollama model list (write in terminal: "ollama list" without quote) and model names. Use the correct model name.

my_models=[
    "codegemma:7b",
    "codestral:22b",
    "codellama:34b",
    "codellama:7b",
    "command-r:35b",
    "deepseek-coder:1.3b",
    "Deepseek-coder-v2:16b",
    "dolphin-mistral:7b",
    "gemma2:27b",
    "huihui_ai/qwq-abliterated:latest",
    "huihui_ai/qwq-fusion:latest",
    "llama2:7b",
    "Llama3:8b",
    "llama3.1:8b",
    "llama3.2:3b",
    "llava-llama3:8b",
    "marco-o1:7b",
    "mistral:latest",
    "mistral-nemo:12b",
    "openchat:7b",
    "orca-mini:3b",
    "phi:2.7b",
    "phi3:3.8b",
    "qwen:4b",
    "qwen2:7b",
    "qwen2.5:7b",
    "qwq:latest",
    "starcoder:3b",
    "starcoder2:3b",
    "TinyLlama:1.1b",
    "wizardlm2:7b",
    "yi:6b",
    "zephyr:7b"
]

very_large_models = [
"llama3.3:latest",
"llama2:70b",
"codellama:70b"]

default_model = "llama2:7b"

#replace with your host IP or use localhost. Make sure the correct port number of ollama server.
ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [60]:
def run_LLM_conflict(flow_json_1, flow_json_2, model):
    
    system_prompt = CONFLICT_PROMPT_RYU
    count = 0
    while True:
        count+=1
        try:
            time.sleep(0.1)

            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system=system_prompt,
                prompt=f"Flow 1:\n{json.dumps(flow_json_1, indent=2)}\n\nFlow 2:\n{json.dumps(flow_json_2, indent=2)}",
                format='json'
            )

            output = response['response'].strip()

            response_json = json.loads(output)

            if 'conflict_status' not in response_json:
                #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                break
            else:
                valid_conflict_response = True
                conflict_status = response_json.get('conflict_status', 0)
                # Ensure conflict_status is an integer
                if isinstance(conflict_status, str):
                    conflict_status = int(conflict_status)

                return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

        except Exception as e:
            print("Exception found for model: ",model, ". Reason: ", e)
            sys.stdout.flush()
            if(count<15):
                continue
            else:
                print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                break               
    
    return False, None, None

In [61]:
def safe_load_flow_json(flow_str, index_number):
    try:
        return ast.literal_eval(flow_str)
    except Exception as e:
        print(f"[Index {index_number}] Error parsing flow JSON from excel using ast.literal_eval: {e}")
        return None


In [62]:
df = pd.read_excel("FlowConflict-Ryu.xlsx")

summary_rows = []

for model in my_models:
    TP = FP = TN = FN = 0
    times = []

    for idx, row in df.iterrows():

        index_number = row["Sl."]
        flow_rule_1_json = safe_load_flow_json(row["Ryu Flow Rule 1"], index_number)
        flow_rule_2_json = safe_load_flow_json(row["Ryu Flow Rule 2"], index_number)

        ground_truth_conflict = row["Conflicting"].strip().lower()  # 'yes' or 'no'
        gt_conflict = (ground_truth_conflict == 'yes')

        if (flow_rule_1_json is None) or (flow_rule_2_json is None):
            continue

        start_conflict_time = time.time()
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(
            flow_rule_1_json, flow_rule_2_json, model=model)
        conflict_duration = time.time() - start_conflict_time
        times.append(conflict_duration)

        # Only score if a valid result was returned
        if conflict_status == 2 or not valid_conflict_response:
            continue
        detected_conflict = (conflict_status == 1)

        # Update counts
        if detected_conflict and gt_conflict:
            TP += 1
        elif detected_conflict and not gt_conflict:
            FP += 1
        elif not detected_conflict and not gt_conflict:
            TN += 1
        elif not detected_conflict and gt_conflict:
            FN += 1

    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    avg_time = sum(times) / len(times) if times else 0

    summary_rows.append([
        model, TP, FP, TN, FN, round(accuracy, 4), round(precision, 4),
        round(recall, 4), round(f1, 4), round(fpr, 4), round(avg_time, 4)
    ])

# ---- Write summary CSV ----
with open("Ryu_conflict_detection_model_summaries.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "TP", "FP", "TN", "FN", "Accuracy", "Precision", "Recall", "F1", "FPR", "Avg Time (s)"])
    for row in summary_rows:
        writer.writerow(row)

print("Processing completed. Model summaries saved to CSV.")

Exception found for model:  orca-mini:3b . Reason:  Expecting ',' delimiter: line 3 column 182 (char 209)
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Reason:  'conflict_explanation'
Exception found for model:  TinyLlama:1.1b . Re